In [ ]:
pip install cobra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 8.9 MB/s 
     |████████████████████████████████| 6.8 MB 35.0 MB/s 
     |████████████████████████████████| 2.3 MB 49.4 MB/s 
     |████████████████████████████████| 147 kB 65.1 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 44 kB 3.4 MB/s 
     |████████████████████████████████| 235 kB 69.5 MB/s 
     |████████████████████████████████| 109 kB 46.5 MB/s 
     |████████████████████████████████| 68 kB 6.8 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 80 kB 9.8 MB/s 
     |████████████████████████████████| 51 kB 7.1 MB/s 
     |████████████████████████████████| 546 kB 58.9 MB/s 


In [ ]:
import cobra 


In [ ]:
help(cobra)

Help on package cobra:

NAME
    cobra

PACKAGE CONTENTS
    core (package)
    exceptions
    flux_analysis (package)
    io (package)
    manipulation (package)
    medium (package)
    sampling (package)
    summary (package)
    util (package)

SUBMODULES
    data

VERSION
    0.25.0

AUTHOR
    The cobrapy core development team.

FILE
    /usr/local/lib/python3.7/dist-packages/cobra/__init__.py




In [ ]:
glc = cobra.Metabolite(id='glc',name='D-glucose',
                       formula= 'C6H12O6', charge=None)
pyr =cobra.Metabolite(id='pyr', name='Pyruvate', 
                      formula='C3H3O3')
pep =cobra.Metabolite(id='g6p', name='Phosphoenolpyruvate',
                      formula='C3H2O6P')
g6p =cobra.Metabolite(id='g6p',name='D-glucose 6-phosphate',
                      formula='C6H11O9P')
#construct a COBRA Reaction
GLCpts = cobra.Reaction(
    id='GLCpts',name='D-glucose Transport via PEP:Pyr Pts',
    lower_bound=0. , upper_bound=1000.
)

# Specify the reaction stoichiometry 
GLCpts.add_metabolites (
    {glc: -1.0, pep: -1.0, g6p: 1.0, pyr: 1.0})

#Ensure the reaction is mass-balanced.

assert GLCpts.check_mass_balance() == {}

#Add a number of other reactions to complete the model.
#Note : These reactions are not mass-balanced
upper_glycol = cobra.Reaction(id='upg')
lower_glycol = cobra.Reaction(id='lwg')
atp_maint    = cobra.Reaction(id='atpm')
pyr_kin      = cobra.Reaction(id='pyk')


In [ ]:
atp =cobra.Metabolite('atp', formula='C10H16N5O13P3')
adp =cobra.Metabolite('adp', formula='C10H15N5O10P2')
co2 =cobra.Metabolite('co2', formula='CO2')
h2o =cobra.Metabolite('h2o', formula='H2O')


In [ ]:

upper_glycol.add_metabolites({g6p :-1, pyr:1, pep:1, atp:1, adp:-1})
lower_glycol.add_metabolites({pyr: -1,adp:  -1, co2: 3,atp: 3})
atp_maint.add_metabolites({atp:-1,h2o:-1,adp: 1})
pyr_kin.add_metabolites({pyr: -1,adp: -1,atp: 1,pep: 1})




In [ ]:
#Create the cobrea model object and add each other reaction.
#Note, metabolites are added automaticallly
test_model=cobra.Model('test')
test_model.add_reactions([upper_glycol,lower_glycol, atp_maint, GLCpts])

#Add exchanges for the boundary species
co2_bound = test_model.add_boundary(test_model.metabolites.co2)
glc_bound = test_model.add_boundary(test_model.metabolites.glc)
glc_bound.lower_bound = -10

RuntimeError: ignored

In [ ]:
#Specify a model objective and optimize the model using FBA
test_model.objective = test_model.reactions.atpm
test_model.optimize()

,fluxes,reduced_costs
upg,0.0,0.0
lwg,0.0,0.0
atpm,0.0,0.0
GLCpts,0.0,0.0
